In [1]:
from mpmath import degrees, radians
from sympy import *
from sympy.matrices import Matrix, eye

In [2]:
def col(input_tuple, h=0):
    "option: h=1 will output a homogenous coordinate column vector"
    matrixArg = [[item] for item in input_tuple]
    if h: matrixArg.append([1])
    return Matrix(matrixArg)

In [3]:
e_x = col((1, 0, 0))
e_y = col((0, 1, 0))
e_z = col((0, 0, 1))

In [6]:
### Create symbols for DH parameters
# General parameters:
# alpha, a, d, q = symbols('alpha, a, d, q')
# Indexed parameters:
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8')  # theta angles
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')
# Enter table of values:
dh = {alpha0: 0,        a0: 0,      d1: 0.75,   #q1: q1,
      alpha1: -pi / 2,  a1: 0.35,   d2: 0,      q2: q2 - pi / 2,
      alpha2: 0,        a2: 1.25,   d3: 0,      #q3: q3,
      alpha3: -pi / 2,  a3: -0.054, d4: 1.50,   #q4: q4,
      alpha4: pi / 2,   a4: 0,      d5: 0,      #q5: q5,
      alpha5: -pi / 2,  a5: 0,      d6: 0,      #q6: q6,
      alpha6: 0,        a6: 0,      d7: 0.303,  q7: 0
      }

In [7]:
def composeRotations(rotation_list):
    "rotation_list contains tuples of form (axis, radian_angle), where axis is an element of {'x', 'y', 'z'}"
    def rot_x(q):
        R_x = Matrix([[ 1,              0,        0],
                      [ 0,        cos(q),   -sin(q)],
                      [ 0,        sin(q),    cos(q)]])
        return R_x

    def rot_y(q):
        R_y =  Matrix([[ cos(q),        0,  sin(q)],
                       [       0,        1,        0],
                       [-sin(q),        0,  cos(q)]])
        return R_y

    def rot_z(q): 
        R_z = Matrix([[ cos(q), -sin(q),        0],
                      [ sin(q),  cos(q),        0],
                      [ 0,            0,        1]])
        return R_z
    
    rMat = eye(3)
    for rotation in rotation_list:
        rotation_axis, rotation_angle = rotation[0], rotation[1]
        rr = eval('rot_{}({})'.format(rotation_axis, rotation_angle))
        rMat = rr * rMat
    return rMat

def TF_matrix(rotation_list=[], translation=(0, 0, 0)):
    "rotation_list formatted for composeRotations(),"
    TFmat = composeRotations(rotation_list)    
    TFmat = TFmat.row_join(col(translation))
    TFmat = TFmat.col_join(Matrix([[0, 0, 0, 1]]))
    return TFmat

def DH_TF(alpha, a, d, q):
    # alpha, a, d, q = twist_angle, link_length, joint_angle, link_offset
    subT_x = TF_matrix([('x', alpha)], (a, 0, 0))
    subT_z = TF_matrix([('z', q)], (0, 0, d))
    return subT_x * subT_z

def dissect_TF(DH_TF_mat):
    rot_mat = DH_TF_mat[0:3, 0:3]
    xlt_mat = DH_TF_mat[0:3, 3]
    return rot_mat, xlt_mat

def rotation_matrix_from(DH_TF_mat):
    return DH_TF_mat[0:3, 0:3]

def translation_column_from(DH_TF_mat):
    return DH_TF_mat[0:3, 3]

def vector_cross_angle(first_vector, second_vector):
    # The vector arguments should be Matrix objects with three elements
    unit_1 = first_vector.normalized()
    unit_2 = second_vector.normalized()
    cross_vector = unit_1.cross(unit_2)
    return N(asin(cross_vector.norm()))

def vector_cross_unit(first_vector, second_vector):
    # The vector arguments should be Matrix objects with three elements
    unit_1 = first_vector.normalized()
    unit_2 = second_vector.normalized()
    cross_vector = unit_1.cross(unit_2)
    return cross_vector.normalized()

In [8]:
# t4, t5, t6 = symbols('t4, t5, t6')
R_abc = simplify(composeRotations([('z', q6), ('x', q5), ('z', q4)]))
R_abc

Matrix([
[-sin(q4)*sin(q6)*cos(q5) + cos(q4)*cos(q6), -sin(q4)*cos(q5)*cos(q6) - sin(q6)*cos(q4),  sin(q4)*sin(q5)],
[ sin(q4)*cos(q6) + sin(q6)*cos(q4)*cos(q5), -sin(q4)*sin(q6) + cos(q4)*cos(q5)*cos(q6), -sin(q5)*cos(q4)],
[                           sin(q5)*sin(q6),                            sin(q5)*cos(q6),          cos(q5)]])

In [9]:
init_printing(use_latex='mathjax')

In [10]:
R_corr = Matrix([
    [0,  0,  1,  0],
    [0, -1,  0,  0],
    [1,  0,  0,  0],
    [0,  0,  0,  1]])

def composeRotations(rotation_list):
    "rotation_list contains tuples of form (axis, radian_angle), where axis is an element of {'x', 'y', 'z'}"
    def rot_x(q):
        R_x = Matrix([[ 1,              0,        0],
                      [ 0,        cos(q),   -sin(q)],
                      [ 0,        sin(q),    cos(q)]])
        return R_x

    def rot_y(q):
        R_y =  Matrix([[ cos(q),        0,  sin(q)],
                       [       0,        1,        0],
                       [-sin(q),        0,  cos(q)]])
        return R_y

    def rot_z(q): 
        R_z = Matrix([[ cos(q), -sin(q),        0],
                      [ sin(q),  cos(q),        0],
                      [ 0,            0,        1]])
        return R_z
    
    rMat = eye(3)
    for rotation in rotation_list:
        rotation_axis, rotation_angle = rotation[0], rotation[1]
        rr = eval('rot_{}({})'.format(rotation_axis, rotation_angle))
        rMat = rr * rMat
    return rMat

R_abc = simplify(composeRotations([('z', q6), ('x', q5), ('z', q4)])) * R_corr[0:3, 0:3]
r11 = R_abc[0, 0]
r12 = R_abc[0, 1]
r21 = R_abc[1, 0]
r22 = R_abc[1, 1]
term1 = simplify((r21*r21) + (r22*r22))
term2 = simplify((r12*r12) + (r11*r11))
sin_q5 = sqrt(-1 * (term1 + term2 - 2))
R_abc

⎡sin(q₄)⋅sin(q₅)   sin(q₄)⋅cos(q₅)⋅cos(q₆) + sin(q₆)⋅cos(q₄)  -sin(q₄)⋅sin(q₆)
⎢                                                                             
⎢-sin(q₅)⋅cos(q₄)  sin(q₄)⋅sin(q₆) - cos(q₄)⋅cos(q₅)⋅cos(q₆)  sin(q₄)⋅cos(q₆) 
⎢                                                                             
⎣    cos(q₅)                   -sin(q₅)⋅cos(q₆)                            sin

⋅cos(q₅) + cos(q₄)⋅cos(q₆)⎤
                          ⎥
+ sin(q₆)⋅cos(q₄)⋅cos(q₅) ⎥
                          ⎥
(q₅)⋅sin(q₆)              ⎦

In [11]:
theta5 = atan2(sin_q5, R_abc[2, 2])
theta5

     ⎛   _____________________________________________________________________
     ⎜  ╱                                              2                      
atan2⎝╲╱  - (sin(q₄)⋅sin(q₆) - cos(q₄)⋅cos(q₅)⋅cos(q₆))  - (sin(q₄)⋅cos(q₅)⋅co

______________________________________________________________________        
                        2      2        2          2        2                 
s(q₆) + sin(q₆)⋅cos(q₄))  - sin (q₄)⋅sin (q₅) - sin (q₅)⋅cos (q₄) + 2 , sin(q₅

         ⎞
         ⎟
)⋅sin(q₆)⎠

In [12]:
# term2 = simplify((r12*r12) + (r11*r11))

In [13]:
simplify((r12*r12) + (r11*r11))

                                           2      2        2    
(sin(q₄)⋅cos(q₅)⋅cos(q₆) + sin(q₆)⋅cos(q₄))  + sin (q₄)⋅sin (q₅)

In [14]:
# %%timeit
# alpha, a, d, q = symbols('alpha, a, d, q')
# # Indexed parameters:
# q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8')  # theta angles
# d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
# a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
# alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')
# # Enter table of values:
# dh = {alpha0: 0,        a0: 0,      d1: 0.75,   q1: q1,
#       alpha1: -pi / 2,  a1: 0.35,   d2: 0,      q2: q2 # - pi / 2,
#       alpha2: 0,        a2: 1.25,   d3: 0,      q3: q3,
#       alpha3: -pi / 2,  a3: -0.054, d4: 1.50,   q4: q4,
#       alpha4: pi / 2,   a4: 0,      d5: 0,      q5: q5,
#       alpha5: -pi / 2,  a5: 0,      d6: 0,      q6: q6,
#       alpha6: 0,        a6: 0,      d7: 0.303,  q7: 0
#       }

In [15]:
# %%lsmagic

In [16]:
def TF_matrix(alpha, a, d, q):
    TF = Matrix([
        [           cos(q),             -sin(q),             0,               a],
        [sin(q)*cos(alpha),   cos(q)*cos(alpha),   -sin(alpha),   -d*sin(alpha)],
        [sin(q)*sin(alpha),   cos(q)*sin(alpha),    cos(alpha),    d*cos(alpha)],
        [                0,                   0,             0,               1]])
    return TF

In [17]:
T0_1 = 
T1_2 = 

SyntaxError: invalid syntax (<ipython-input-17-024c89e5e211>, line 1)

In [20]:
# %%timeit
T0_1 = Matrix([
    [cos(q1),              -sin(q1),            0,              a0],
    [sin(q1)*cos(alpha0),   cos(q1)*cos(alpha0),  -sin(alpha0),    -d1*sin(alpha0)],
    [sin(q1)*sin(alpha0),   cos(q1)*sin(alpha0),  cos(alpha0),     d1*cos(alpha0)],
    [0,                   0,                  0,              1]])
T0_1 = T0_1.subs(dh)

T1_2 = Matrix([
    [cos(q2),              -sin(q2),            0,              a1],
    [sin(q2)*cos(alpha1),   cos(q2)*cos(alpha1),  -sin(alpha1),    -d2*sin(alpha1)],
    [sin(q2)*sin(alpha1),   cos(q2)*sin(alpha1),  cos(alpha1),     d2*cos(alpha1)],
    [0,                   0,                  0,              1]])
T1_2 = T1_2.subs(dh)

T2_3 = Matrix([
    [cos(q3),              -sin(q3),            0,              a2],
    [sin(q3)*cos(alpha2),   cos(q3)*cos(alpha2),  -sin(alpha2),    -d3*sin(alpha2)],
    [sin(q3)*sin(alpha2),   cos(q3)*sin(alpha2),  cos(alpha2),     d3*cos(alpha2)],
    [0,                   0,                  0,              1]])
T2_3 = T2_3.subs(dh)

T3_4 = Matrix([
    [cos(q4),               -sin(q4),               0,                  a3],
    [sin(q4)*cos(alpha3),   cos(q4)*cos(alpha3),    -sin(alpha3),       -d4*sin(alpha3)],
    [sin(q4)*sin(alpha3),   cos(q4)*sin(alpha3),    cos(alpha3),     d4*cos(alpha3)],
    [0,                     0,                      0,              1]])
T3_4 = T3_4.subs(dh)

T4_5 = Matrix([
    [cos(q5),               -sin(q5),            0,                  a4],
    [sin(q5)*cos(alpha4),   cos(q5)*cos(alpha4), -sin(alpha4),     -d5*sin(alpha4)],
    [sin(q5)*sin(alpha4),   cos(q5)*sin(alpha4), cos(alpha4),      d5*cos(alpha4)],
    [0,                     0,                   0,              1]])
T4_5 = T4_5.subs(dh)

T5_6 = Matrix([
    [cos(q6),               -sin(q6),               0,               a5],
    [sin(q6)*cos(alpha5),   cos(q6)*cos(alpha5),    -sin(alpha5),    -d6*sin(alpha5)],
    [sin(q6)*sin(alpha5),   cos(q6)*sin(alpha5),    cos(alpha5),     d6*cos(alpha5)],
    [0,                     0,                      0,               1]])
T5_6 = T5_6.subs(dh)

T6_7 = Matrix([
    [cos(q7),               -sin(q7),               0,              a6],
    [sin(q7)*cos(alpha6),   cos(q7)*cos(alpha6),    -sin(alpha6),   -d7*sin(alpha6)],
    [sin(q7)*sin(alpha6),   cos(q7)*sin(alpha6),    cos(alpha6),    d7*cos(alpha6)],
    [0,                     0,                      0,              1]])
T6_7 = T6_7.subs(dh)

In [21]:
R0_1 = T0_1[0:3, 0:3]
R1_2 = T1_2[0:3, 0:3]
R2_3 = T2_3[0:3, 0:3]
R3_4 = T3_4[0:3, 0:3]
R4_5 = T4_5[0:3, 0:3]
R5_6 = T5_6[0:3, 0:3]

In [22]:
R0_6 = simplify(((((R0_1 * R1_2) * R2_3) * R3_4) * R4_5) * R5_6)
R0_6

⎡((sin(q₁)⋅sin(q₄) + sin(q₂ + q₃)⋅cos(q₁)⋅cos(q₄))⋅cos(q₅) + sin(q₅)⋅cos(q₁)⋅c
⎢                                                                             
⎢((sin(q₁)⋅sin(q₂ + q₃)⋅cos(q₄) - sin(q₄)⋅cos(q₁))⋅cos(q₅) + sin(q₁)⋅sin(q₅)⋅c
⎢                                                                             
⎣                                -(sin(q₅)⋅sin(q₂ + q₃) - cos(q₄)⋅cos(q₅)⋅cos(

os(q₂ + q₃))⋅cos(q₆) - (-sin(q₁)⋅cos(q₄) + sin(q₄)⋅sin(q₂ + q₃)⋅cos(q₁))⋅sin(q
                                                                              
os(q₂ + q₃))⋅cos(q₆) - (sin(q₁)⋅sin(q₄)⋅sin(q₂ + q₃) + cos(q₁)⋅cos(q₄))⋅sin(q₆
                                                                              
q₂ + q₃))⋅cos(q₆) - sin(q₄)⋅sin(q₆)⋅cos(q₂ + q₃)                              

₆)  -((sin(q₁)⋅sin(q₄) + sin(q₂ + q₃)⋅cos(q₁)⋅cos(q₄))⋅cos(q₅) + sin(q₅)⋅cos(q
                                                                              
)   -((sin(q₁)⋅sin(q₂ + q₃)⋅cos(q₄) - sin(q₄)⋅cos(

In [23]:
R0_3 = simplify((R0_1 * R1_2) * R2_3)
R0_3

⎡sin(q₂ + q₃)⋅cos(q₁)  cos(q₁)⋅cos(q₂ + q₃)  -sin(q₁)⎤
⎢                                                    ⎥
⎢sin(q₁)⋅sin(q₂ + q₃)  sin(q₁)⋅cos(q₂ + q₃)  cos(q₁) ⎥
⎢                                                    ⎥
⎣    cos(q₂ + q₃)         -sin(q₂ + q₃)         0    ⎦

In [24]:
invR03 = simplify(R0_3.inv(method="LU"))
print(repr(invR03))

Matrix([
[sin(q2 + q3)*cos(q1), sin(q1)*sin(q2 + q3),  cos(q2 + q3)],
[cos(q1)*cos(q2 + q3), sin(q1)*cos(q2 + q3), -sin(q2 + q3)],
[            -sin(q1),              cos(q1),             0]])


In [ ]:
# # %%timeit
# print("Method 1:\n")
# %time T0_2 = simplify(T0_1 * T1_2)
# %time T0_3 = simplify(T0_2 * T2_3)
# %time T0_4 = simplify(T0_3 * T3_4)
# %time T0_5 = simplify(T0_4 * T4_5)
# %time T0_6 = simplify(T0_5 * T5_6)
# %time T0_7 = simplify(T0_6 * T6_7)
# # print("Method 2:\n")
# %time T0_3_alt = simplify(T0_1 * T1_2 * T2_3)
# %time T3_7_alt = simplify(T3_4 * T4_5 * T5_6 * T6_7)
# %time T0_7_alt = T0_3_alt * T3_7_alt
# %time T0_7_alt = simplify(T0_7_alt)


# # %time T0_4 = simplify(T0_3 * T3_4)
# # %time T0_5 = simplify(T0_4 * T4_5)
# # %time T0_6 = simplify(T0_5 * T5_6)
# # %time T0_7 = simplify(T0_6 * T6_7)

In [25]:
T0_2 = simplify(T0_1 * T1_2)
T0_3 = simplify(T0_2 * T2_3)
T0_4 = simplify(T0_3 * T3_4)
T0_5 = simplify(T0_4 * T4_5)
T0_6 = simplify(T0_5 * T5_6)
T0_7 = simplify(T0_6 * T6_7)

R0_3 = T0_3[0:3, 0:3]
wc = T0_4[0:3, 3]

In [26]:
# R0_6 = T0_6[0:3, 0:3]
# R0_6

In [27]:
R0_6 == R0_6_alt

NameError: name 'R0_6_alt' is not defined

In [ ]:
fk_EE = simplify(T0_7[0:3, 3])
print(fk_EE)

In [ ]:
# T0_4 = simplify(((T0_1 * T1_2) * T2_3) * T3_4)
# T0_4

In [ ]:
# T0_4 = simplify((simplify(T0_1 * T1_2) * T2_3) * T3_4))
# print(repr(T0_4))

In [ ]:
# R0_3 = T0_3[0:3, 0:3]
# wc = T0_4[0:3, 3]
# R0_3

In [ ]:
# print(repr(wc))

In [ ]:
# %%timeit
# T3_7 = simplify(T3_4 * T4_5) * simplify(T5_6 * T6_7)
# T0_7 = simplify(T0

In [ ]:
qdict = {q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}

In [ ]:
# T0_7 = simplify(T0_3 * T3_7)

In [ ]:
# ee = T0_7[0:3, 3]
# ee.subs(qdict)

In [ ]:
# T0_7 = simplify(T0_3 * (simplify(T3_4 * T4_5) * simplify(T5_6 * T6_7)))
# T0_7 = simplify(T0_7)

In [ ]:
# print(repr(T0_3

In [ ]:
# T0_7

In [ ]:
# print(repr(R0_3))

In [ ]:
# %%timeit
Rinv0_3 = simplify(R0_3.inv(method="LU"))
print(repr(Rinv0_3))

In [ ]:
Rinv0_3 == invR03_alt

In [ ]:
# %%timeit
# R_z = Matrix([[ cos(pi),     -sin(pi),        0,        0],
#               [ sin(pi),      cos(pi),        0,        0],
#               [       0,            0,        1,        0],
#               [       0,            0,        0,        1]
#               ]) 
# R_y = Matrix([[ cos(-pi/2),        0,  sin(-pi/2),        0],
#               [          0,        1,           0,        0],
#               [-sin(-pi/2),        0,  cos(-pi/2),        0],
#               [          0,        0,           0,        1]
#               ])
# R_corr = N(R_z * R_y)

In [ ]:
# %%timeit
R_corr = Matrix([
    [0,  0,  1,  0],
    [0, -1,  0,  0],
    [1,  0,  0,  0],
    [0,  0,  0,  1]])

In [ ]:
R3_6_alt = simplify(Rinv0_3 * R0_6_alt)
R3_6_alt

In [9]:
T_total = simplify(T0_7 * R_corr)

In [ ]:
T_total

In [ ]:
T_total = simplify(T_total)
# T_total

In [47]:
R3_6 = simplify(Rinv0_3 * R0_6)
R3_6

⎡                                                                             
⎢-sin(q₄)⋅sin(q₆) + cos(q₄)⋅cos(q₅)⋅cos(q₆)  -sin(q₄)⋅cos(q₆) - sin(q₆)⋅cos(q₄
⎢                                                                             
⎢                                                                             
⎢             sin(q₅)⋅cos(q₆)                             -sin(q₅)⋅sin(q₆)    
⎢                                                                             
⎣-sin(q₄)⋅cos(q₅)⋅cos(q₆) - sin(q₆)⋅cos(q₄)  sin(q₄)⋅sin(q₆)⋅cos(q₅) - cos(q₄)

           sin(q₄ - q₅)   sin(q₄ + q₅)⎤
)⋅cos(q₅)  ──────────── - ────────────⎥
                2              2      ⎥
                                      ⎥
                     cos(q₅)          ⎥
                                      ⎥
⋅cos(q₆)         sin(q₄)⋅sin(q₅)      ⎦

In [58]:
simplify(sin(q4 - q5)/2 - sin(q4 + q5)/2)

-sin(q₅)⋅cos(q₄)

In [65]:
R3_6_edit = Matrix([
    [-sin(q4)*sin(q6) + cos(q4)*cos(q5)*cos(q6), -sin(q4)*cos(q6) - sin(q6)*cos(q4)*cos(q5), -cos(q4)*sin(q5)],
    [                           sin(q5)*cos(q6),                           -sin(q5)*sin(q6),          cos(q5)],
    [-sin(q4)*cos(q5)*cos(q6) - sin(q6)*cos(q4),  sin(q4)*sin(q6)*cos(q5) - cos(q4)*cos(q6),  sin(q4)*sin(q5)]])
R3_6_edit

⎡-sin(q₄)⋅sin(q₆) + cos(q₄)⋅cos(q₅)⋅cos(q₆)  -sin(q₄)⋅cos(q₆) - sin(q₆)⋅cos(q₄
⎢                                                                             
⎢             sin(q₅)⋅cos(q₆)                             -sin(q₅)⋅sin(q₆)    
⎢                                                                             
⎣-sin(q₄)⋅cos(q₅)⋅cos(q₆) - sin(q₆)⋅cos(q₄)  sin(q₄)⋅sin(q₆)⋅cos(q₅) - cos(q₄)

)⋅cos(q₅)  -sin(q₅)⋅cos(q₄)⎤
                           ⎥
               cos(q₅)     ⎥
                           ⎥
⋅cos(q₆)   sin(q₄)⋅sin(q₅) ⎦

In [66]:
r11 = R3_6_edit[0, 0]
r12 = R3_6_edit[0, 1]
r13 = R3_6_edit[0, 2]
r21 = R3_6_edit[1, 0]
r22 = R3_6_edit[1, 1]
r23 = R3_6_edit[1, 2]
r31 = R3_6_edit[2, 0]
r32 = R3_6_edit[2, 1]
r33 = R3_6_edit[2, 2]

In [1]:
# simplify(r11*r11 + r12*r12)

NameError: name 'simplify' is not defined

In [60]:
simplify(r22/r21)

-tan(q₆)

In [61]:
theta6 = -atan2(r22, r21)

In [68]:
simplify(r33/r13)

-tan(q₄)

In [69]:
theta4 = -atan2(r33, r13)

In [ ]:
sin_q5 = sqrt(-1 * (simplify(r11*r11 + r12*r12) + simplify(r31*r31 + r32*r32) - 2))
theta5 = atan2(sin_q5, r23)

In [ ]:
# r11 = R3_6_edit[0, 0]
# r12 = R3_6_edit[0, 1]
# r13 = R3_6_edit[0, 2]
# r21 = R3_6_edit[1, 0]
# r22 = R3_6_edit[1, 1]
# r23 = R3_6_edit[1, 2]
# r31 = R3_6_edit[2, 0]
# r32 = R3_6_edit[2, 1]
# r33 = R3_6_edit[2, 2]

# theta4 = -atan2(r33, r13)

# sin_q5 = sqrt(-1 * (simplify(r11*r11 + r12*r12) + simplify(r31*r31 + r32*r32) - 2))
# theta5 = atan2(sin_q5, r23)

# theta6 = -atan2(r22, r21)

In [3]:
# j4_correction = 
N(atan2(0.054, 1.50))

0.0359844600820516

In [19]:
r, p, y = symbols('r p y')


R_x = Matrix([[ 1,            0,       0],
              [ 0,       cos(r), -sin(r)],
              [ 0,       sin(r),  cos(r)]])

R_y = Matrix([[ cos(p),       0,  sin(p)],
              [      0,       1,       0],
              [-sin(p),       0,  cos(p)]])

R_z = Matrix([[ cos(y), -sin(y),       0],
              [ sin(y),  cos(y),       0],
              [ 0,            0,       1]])

R_EE = R_z * R_y * R_x

R_error = R_z.subs(y, radians(180)) * R_y.subs(p, radians(-90))

R_corr = Matrix([
            [ 0,  0,  1],
            [ 0, -1,  0],
            [ 1,  0,  0]])

R_EE = R_EE * R_error

# R_EE = R_EE.subs({r: roll, p: pitch, y: yaw})

In [21]:
R_EE = R_z * R_y * R_x
simplify(R_EE * R_corr)

Matrix([
[sin(p)*cos(r)*cos(y) + sin(r)*sin(y), -sin(p)*sin(r)*cos(y) + sin(y)*cos(r), cos(p)*cos(y)],
[sin(p)*sin(y)*cos(r) - sin(r)*cos(y), -sin(p)*sin(r)*sin(y) - cos(r)*cos(y), sin(y)*cos(p)],
[                       cos(p)*cos(r),                        -sin(r)*cos(p),       -sin(p)]])